# Signing Demo for Web3 Engineers

## \[Note on use\] Reset, keeping build artifacts

Run the below cell or commands in a shell to "reset" the kernel, keeping build artifacts:

In [3]:
:clear // Clear all state, **keeping compilation cache**, use this over a kernel restart when possible. You will need to re-run the :deps to have them loaded into state. 
:last_compile_dir // Show where the target is for cargo for this kernel, in case you want to recover these

"/tmp/.tmp3GYuJy"


Set the env var `EVCXR_TMPDIR=<the output of :last_compile_dir>` to get the REPL to use this dir.

```sh
# Launching jupyter-lab
EVCXR_TMPDIR=<dir from :last_compile_dir> jupyter-lab

# Using Evcxr in a shell
EVCXR_TMPDIR=<dir from :last_compile_dir> evcxr
```

## Notebook Kernel & REPL Environment Setup

The below should be run at kernel startup before you start.
All dependencies that we need to build _before_ anything else in this notebook will work.

Instead of rebuilding see the `Reset, keeping build artifacts` section to clear REPL state only.


In [4]:
:dep sp-core = { version = "7.0.0", git = "https://github.com/paritytech/substrate.git", branch = "polkadot-v0.9.36" }

// Loading & building dependencies crates here takes *a while*! 
// Run this while you move on to the readings below.
// NOTE: A kernel restart removes all target artifacts!
// ONLY restart only if explicitly needed.

## Digital Signatures

Here we demonstrate a few parts of the [Substrate Primitives (`sp_core`)](https://paritytech.github.io/substrate/master/sp_core/index.html) library for interacting with keys and signatures.

In [5]:
use sp_core::{
	blake2_256,
	crypto::{Derive, Ss58Codec, Ss58AddressFormatRegistry},
	DeriveJunction,
	hexdisplay::HexDisplay,
	Pair as _,
	sr25519::{Pair, Public},
};

## Key Generation

In [6]:
// Generate a secret key.
let (pair, mnemonic, _) = Pair::generate_with_phrase(None);

In [7]:
// Reveal your Secret Seed Phrase
mnemonic

"hawk false thought wife devote close night pizza cabin novel across melt"

In [8]:
// Derive the public key.
let pk = pair.public();

In [9]:
// Print public key as raw bytes
pk.0

[38, 16, 19, 15, 113, 123, 87, 121, 148, 236, 175, 26, 71, 48, 211, 188, 209, 16, 136, 10, 144, 136, 122, 166, 43, 172, 47, 201, 201, 42, 162, 38]

In [10]:
// Print public key hex encoded 
<HexDisplay>::from(&pk.0)

2610130f717b577994ecaf1a4730d3bcd110880a90887aa62bac2fc9c92aa226

## Signatures

In [11]:
// Sign a message.
let msg = b"Welcome to Polkadot Blockchain Academy!";
let sig = pair.sign(&msg[..]);

In [12]:
// Message:
std::str::from_utf8(&msg[..]).unwrap()

"Welcome to Polkadot Blockchain Academy!"

In [13]:
// Signature, hex encoded:
&sig

c6ff39639ec55a975391e222254bac992b2d95cc5393c75444c45bdcb812af04468b549b81c1edbb9c4b06c683353dd23636e2199096553ddb40573c8cd2508d

In [14]:
// Verify the sig. `assert!` will not panic here with a valid sig
assert!(Pair::verify(&sig, &msg[..], &pk));

In [15]:
// Alter the message. `assert!` will panic on msg, here we assert it's fails.
let tampered = b"Welcome-to-Polkadot-Blockchain-Academy!";
assert!(!Pair::verify(&sig, &tampered[..], &pk));

### Signing & Hashes

The signature process can be resource intensive with large data. We can use a *cryptographic hash function* of the data, which is generally a fixed small size and then sign that hash. This signature, the hashing protocol, and the original data is then used to be as valid to us as the signature on the original data without as much resource use.  


In [16]:
// Signing a message hash example, instead of the full msg.
let long_msg =
	b"The term \"Web3\" (AKA Web 3.0) was coined in 2014 by Ethereum co-founder Gavin Wood, is an idea for a new iteration of the World Wide Web which incorporates concepts such as decentralization, blockchain technologies, and token-based economics.";
let lm_hash = blake2_256(&long_msg[..]);

let sig_lm_hash = pair.sign(&lm_hash[..]);

In [17]:
// Hex print hash & signature
<HexDisplay>::from(&lm_hash)

652c97cbbae2caeb52556680c55096c6a9e9aca8318461f1cb4e74019833ed32

In [18]:
// Verify the signature from the same hash of the original message.
assert!(Pair::verify(&sig_lm_hash, blake2_256(&long_msg[..]), &pk));

## Hard Derivation

In [19]:
// Derive new key pairs from the original mnemonic using `//polkadot`.
let pair_polkadot = Pair::from_string(&format!("{}//polkadot", &mnemonic), None);
let pk_polkadot = pair_polkadot.unwrap().public();

In [20]:
// Polkadot Public Key, Hex
<HexDisplay>::from(&pk_polkadot.0)

8a780f44284e9b2137ffa1ab94b05ba91cb903976995298e7aab04ed54219956

In [21]:
// Polkadot Public Key, SS58 for Polkadot
&pk_polkadot.to_ss58check_with_version(Ss58AddressFormatRegistry::PolkadotAccount.into())

"148ZGrDNoUxCXk1SzL8SspRsxqW5Xtb7W9AxkhdhTFgQ7QEy"

In [22]:
// Derive new key pairs from the original mnemonic using `//kusama`.
let pair_kusama = Pair::from_string(&format!("{}//kusama", &mnemonic), None);
let pk_kusama = pair_kusama.unwrap().public();

In [23]:
// Kusama Public Key, Hex
<HexDisplay>::from(&pk_kusama.0)

fc760ea0f7e45b2c4e1b0624cf953b8de471811a57e73db7514e387e6c78e573

In [24]:
// Kusama Public Key, SS58 for Polkadot
&pk_kusama.to_ss58check_with_version(Ss58AddressFormatRegistry::KusamaAccount.into())

"JHLewmVTEEzXwARwCKyFh4syXK98SFUPNQg6YDorRMP3ELx"

## SS58 Tool

https://polkadot.subscan.io/tools/ss58_transform

This handy tool can be used to cross check a raw hex encoded pubkey, and see all SS58 variants for them.

## Soft Derivation

In [25]:
// Derive a soft path on the Polkadot key.
let pair_polkadot_zero = Pair::from_string(&format!("{}//polkadot/0", &mnemonic), None);
let pk0_from_secret = pair_polkadot_zero.unwrap().public();

In [26]:
// Polkadot Soft-Derived Public Key (from secret)
&pk0_from_secret.to_ss58check_with_version(Ss58AddressFormatRegistry::PolkadotAccount.into())

"123MSN6RKzp8Ha16LmajgeUKJ58YAhPV2UBhAhR5Yswjv2zg"

In [27]:
// Derive a soft path only using the _public key_ of the //polkadot pair.
let pk_polkadot: Public = Public(pk_polkadot.0);
let path0 = vec![DeriveJunction::soft(0u8)];
let pk0_from_public = pk_polkadot.derive(path0.into_iter()).unwrap();

In [28]:
// Polkadot Soft-Derived Public Key (from public only)
&pk0_from_public.to_ss58check_with_version(Ss58AddressFormatRegistry::PolkadotAccount.into())

"123MSN6RKzp8Ha16LmajgeUKJ58YAhPV2UBhAhR5Yswjv2zg"

In [29]:
// Keys are identical.
assert_eq!(pk0_from_secret, pk0_from_public)

()